# Microlithography Interactive Explorer – Final Version

All improvements included:
- Exact vector polygons (no extraction)
- Precomputed aerials
- Mask inversion
- Mack + simple resist tuning
- nm scales + scalebars
- origin='lower' (no upside-down patterns)

In [1]:
!pip install -q torch torchvision matplotlib scipy numpy ipywidgets

import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.ndimage import gaussian_filter, distance_transform_edt
import ipywidgets as widgets
from IPython.display import display, clear_output
import math

# Device
device = torch.device('mps' if torch.backends.mps.is_available() else
                      'cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

RAW_TRAINING_DATA_PATH = "data/litho_dataset_training.npz"
ALL_PATCHES_DATA_PATH = "data/litho_dataset_all_patches.npz"
SAMPLED_PATCHES_DATA_PATH = "data/litho_dataset_sampled_patches.npz"

grid_size = 256
pixel_size_nm = 5.0
pad_ratio = 0.3
full_width_nm = grid_size * pixel_size_nm

Device: mps


In [2]:
# PBC helpers
def pad_mask_for_pbc(mask, pad_ratio=0.3):
    H, W = mask.shape
    pad_h = int(H * pad_ratio)
    pad_w = int(W * pad_ratio)
    padded_size = H + 2 * pad_h
    padded = torch.zeros((padded_size, padded_size), device=mask.device)
    padded[pad_h:pad_h+H, pad_w:pad_w+W] = mask
    return padded, pad_h, pad_w, padded_size

def unpad(tensor, pad_h, pad_w):
    H, W = tensor.shape
    return tensor[pad_h:H-pad_h, pad_w:W-pad_w]

In [3]:
# Core simulation
def create_pupil_and_freq(grid_size, na=1.35, wavelength_nm=193.0, pixel_size_nm=5.0):
    lambda_m = wavelength_nm * 1e-9
    f_cutoff = na / lambda_m
    dx = pixel_size_nm * 1e-9
    freq = torch.fft.fftfreq(grid_size, d=1.0, device=device)
    FX, FY = torch.meshgrid(freq, freq, indexing='ij')
    R = torch.sqrt(FX**2 + FY**2)
    r_norm = f_cutoff * dx
    pupil = (R <= r_norm).cfloat()
    return pupil, FX, FY

def create_source_points():
    x = torch.linspace(-1, 1, 40, device=device)
    y = torch.linspace(-1, 1, 40, device=device)
    X, Y = torch.meshgrid(x, y, indexing='ij')
    R = torch.sqrt(X**2 + Y**2).flatten()
    theta = torch.atan2(Y.flatten(), X.flatten())
    points_grid = torch.stack([X.flatten(), Y.flatten()], dim=1)
    r_mask = (R >= 0.3) & (R <= 0.8)
    half_angle = math.radians(30)/2
    centers = [0, math.pi]
    dists = torch.stack([torch.abs((theta - c + math.pi) % (2*math.pi) - math.pi) for c in centers])
    mask = (dists.min(0).values <= half_angle) & r_mask
    return points_grid[mask]

def precompute_socs_kernels(source_points, pupil, FX, FY, num_kernels=80):
    N = pupil.numel()
    S = len(source_points)
    A = torch.zeros((N, S), dtype=torch.cfloat, device=device)
    for i, (sx, sy) in enumerate(source_points):
        phase = torch.exp(2j * math.pi * (sx * FX + sy * FY))
        A[:, i] = (pupil * phase).flatten() / math.sqrt(S)
    U, S_eig, _ = torch.svd_lowrank(A.cpu(), q=num_kernels + 20)
    kernels_freq = U[:, :num_kernels].to(device).reshape(pupil.shape[0], pupil.shape[1], num_kernels)
    eigenvalues = (S_eig[:num_kernels] ** 2).to(device)
    return kernels_freq, eigenvalues

def simulate_aerial_socs(mask, kernels_freq, eigenvalues):
    if mask.dim() == 2: mask = mask.unsqueeze(0)
    mask_fft = torch.fft.fft2(mask)
    aerial = torch.zeros_like(mask_fft, dtype=torch.float)
    for k in range(eigenvalues.shape[0]):
        field = torch.fft.ifft2(mask_fft * kernels_freq[:,:,k])
        aerial += eigenvalues[k] * (field.abs() ** 2)
    aerial = aerial.real
    uniform_fft = torch.fft.fft2(torch.ones_like(mask))
    clear = torch.zeros_like(aerial)
    for k in range(eigenvalues.shape[0]):
        field = torch.fft.ifft2(uniform_fft * kernels_freq[:,:,k])
        clear += eigenvalues[k] * (field.abs() ** 2)
    aerial /= (clear.real.mean() + 1e-10)
    return aerial.squeeze(0).clamp_(0)

In [4]:
# Resist models
def mack_levelset_resist(aerial, dose=1.1, dill_C=0.07, r_max=70.0, n_mack=4.0, num_steps=150):
    binary = (aerial > 0.3).cpu().numpy().astype(np.float32)
    dist = (distance_transform_edt(binary) - distance_transform_edt(1-binary)).astype(np.float32)
    phi = torch.from_numpy(dist).to(device)
    phi.clamp_(-20, 20)
    E = dose * aerial
    M = torch.exp(-dill_C * E)
    R = r_max * (1 - M)**n_mack + 0.1
    dt = 0.35
    for _ in range(num_steps):
        xp = torch.roll(phi, -1, 1)
        xm = torch.roll(phi,  1, 1)
        yp = torch.roll(phi, -1, 0)
        ym = torch.roll(phi,  1, 0)
        g_plus  = torch.sqrt(torch.max(xp-phi, torch.zeros_like(phi))**2 + torch.max(yp-phi, torch.zeros_like(phi))**2)
        g_minus = torch.sqrt(torch.max(phi-xm, torch.zeros_like(phi))**2 + torch.max(phi-ym, torch.zeros_like(phi))**2)
        grad = torch.where(phi >= 0, g_plus, g_minus) + 1e-8
        phi.sub_(dt * R * grad)
    return torch.clamp((phi + 20) / 40 * 2.0, 0.0, 2.0)

def simple_resist(aerial_np, thresh=0.25, steep=20.0, diff=2.8, load_sigma=16.0, gain=1.8):
    diffused = gaussian_filter(aerial_np, sigma=diff)
    loading = gaussian_filter(aerial_np, sigma=load_sigma)
    effective = diffused + 0.20 * loading
    res = 1.0 / (1.0 + np.exp(-steep * (effective - thresh)))
    return np.clip(res * gain, 0.0, 2.0)

In [5]:
# Structure generators – return (mask, polygons)
def generate_L(size=256, cd=12, long=140, short=80):
    m = torch.zeros((size,size), device=device)
    cx, cy = size//2, size//2
    # Vertical
    vy1, vy2 = cy - long//2, cy + long//2
    vx1, vx2 = cx - cd//2, cx + cd//2
    m[vy1:vy2, vx1:vx2] = 1
    # Horizontal
    hy1, hy2 = cy + long//2 - cd//2, cy + long//2 + cd//2
    hx1, hx2 = cx - cd//2, cx + short
    m[hy1:hy2, hx1:hx2] = 1
    return m, [[vx1,vy1,vx2,vy2], [hx1,hy1,hx2,hy2]]

def generate_T(size=256, cd=12, stem=140, cross=100):
    m = torch.zeros((size,size), device=device)
    cx, cy = size//2, size//2
    vy1, vy2 = cy - stem//2, cy + stem//2
    vx1, vx2 = cx - cd//2, cx + cd//2
    m[vy1:vy2, vx1:vx2] = 1
    hy1, hy2 = cy - stem//2 - cd//2, cy - stem//2 + cd//2
    hx1, hx2 = cx - cross//2, cx + cross//2
    m[hy1:hy2, hx1:hx2] = 1
    return m, [[vx1,vy1,vx2,vy2], [hx1,hy1,hx2,hy2]]

def generate_line_srafs(size=256, cd=12, len=180, sw=4, ns=2, sp=30):
    m = torch.zeros((size,size), device=device)
    cx, cy = size//2, size//2
    vy1, vy2 = cy - len//2, cy + len//2
    vx1, vx2 = cx - cd//2, cx + cd//2
    m[vy1:vy2, vx1:vx2] = 1
    polys = [[vx1,vy1,vx2,vy2]]
    for s in [-1,1]:
        for k in range(1, ns+1):
            x = cx + s * k * sp
            if 0 <= x < size:
                x1, x2 = x - sw//2, x + sw//2
                m[:, x1:x2] = 1
                polys.append([x1, 0, x2, size])
    return m, polys

def generate_L_srafs(size=256, cd=12, long=140, short=80, sw=4, ns=2, sp=30):
    m, polys = generate_L(size, cd, long, short)
    cx = size//2
    for s in [-1,1]:
        for k in range(1, ns+1):
            x = cx + s * k * sp
            if 0 <= x < size:
                x1, x2 = x - sw//2, x + sw//2
                m[:, x1:x2] = 1
                polys.append([x1, 0, x2, size])
    return m, polys

def generate_vertical_lines(size=256, cd=12, pitch=24, num=8):
    m = torch.zeros((size,size), device=device)
    polys = []
    cx = size//2
    start = cx - (num * pitch)//2
    for i in range(num):
        x = start + i * pitch
        if 0 <= x < size:
            x1, x2 = x, x + cd
            m[:, x1:x2] = 1
            polys.append([x1, 0, x2, size])
    return m, polys

def generate_contacts(size=256, diam=14, pitch=40, grid=5):
    m = torch.zeros((size,size), device=device)
    polys = []
    cx, cy = size//2, size//2
    start_x = cx - (grid * pitch)//2
    start_y = cy - (grid * pitch)//2
    for i in range(grid):
        for j in range(grid):
            x = start_x + i * pitch
            y = start_y + j * pitch
            x1, x2 = x - diam//2, x + diam//2
            y1, y2 = y - diam//2, y + diam//2
            m[y1:y2, x1:x2] = 1
            polys.append([x1, y1, x2, y2])
    return m, polys

def generate_horizontal_grating(size=256, cd=12, pitch=28, num=10):
    m = torch.zeros((size,size), device=device)
    polys = []
    cy = size//2
    start = cy - (num * pitch)//2
    for i in range(num):
        y = start + i * pitch
        if 0 <= y < size:
            y1, y2 = y, y + cd
            m[y1:y2, :] = 1
            polys.append([0, y1, size, y2])
    return m, polys

structures = {
    'L-shape'             : generate_L,
    'L + SRAFs'           : generate_L_srafs,
    'T-junction'          : generate_T,
    'Line + SRAFs'        : generate_line_srafs,
    'Vertical lines'      : generate_vertical_lines,
    'Contacts 5×5'        : generate_contacts,
    'Horizontal grating'  : generate_horizontal_grating,
}

In [6]:
# Precompute aerials + store polygons
padded_size = grid_size + 2 * int(grid_size * pad_ratio)
pupil, FX, FY = create_pupil_and_freq(padded_size)
source_pts = create_source_points()
kernels, evals = precompute_socs_kernels(source_pts, pupil, FX, FY)

precomputed = {}
print("Precomputing aerials...")
for name, gen in structures.items():
    mask, polygons = gen()
    padded, ph, pw, _ = pad_mask_for_pbc(mask)
    ap = simulate_aerial_socs(padded, kernels, evals)
    aerial = unpad(ap, ph, pw)
    precomputed[name] = {
        'mask': mask,
        'aerial': aerial,
        'aerial_np': aerial.cpu().numpy(),
        'polygons': polygons
    }
    print(f"  {name}")
print("Precomputation complete.")

Precomputing aerials...
  L-shape
  L + SRAFs
  T-junction
  Line + SRAFs
  Vertical lines
  Contacts 5×5
  Horizontal grating
Precomputation complete.


In [ ]:
# ────────────────────────────────────────────────
#  Interactive Widget
# ────────────────────────────────────────────────

out = widgets.Output()

dd_structure = widgets.Dropdown(options=list(structures), description='Structure')
cb_invert    = widgets.Checkbox(value=False, description='Invert mask')
cb_poly      = widgets.Checkbox(value=True,  description='Show polygons')
btn_update   = widgets.Button(description='Update', button_style='success')

dose   = widgets.FloatSlider(value=1.1, min=0.5, max=2.0, step=0.05, description='Dose')
dill_c = widgets.FloatSlider(value=0.07, min=0.01, max=0.20, step=0.005, description='Dill C')
r_max  = widgets.FloatSlider(value=70.0, min=20, max=200, step=5, description='R_max')
n_mack = widgets.FloatSlider(value=4.0, min=1, max=10, step=0.25, description='n_Mack')
steps  = widgets.IntSlider(value=150, min=50, max=400, step=25, description='Steps')

thresh   = widgets.FloatSlider(value=0.25, min=0.0, max=0.8, step=0.01, description='Thresh')
steep    = widgets.FloatSlider(value=20.0, min=5.0, max=50.0, step=1.0, description='Steepness')
diff     = widgets.FloatSlider(value=2.8, min=0.5, max=8.0, step=0.2, description='Diff σ')
load_sig = widgets.FloatSlider(value=16.0, min=5.0, max=40.0, step=1.0, description='Load σ')
gain     = widgets.FloatSlider(value=1.8, min=0.5, max=3.0, step=0.1, description='Gain')

model = widgets.ToggleButtons(options=['Mack', 'Simple'], value='Mack')

def update(_):
    with out:
        clear_output(wait=True)
        sel = dd_structure.value
        inv = cb_invert.value
        data = precomputed[sel]
        mask = 1.0 - data['mask'] if inv else data['mask']
        polys = data['polygons'] if cb_poly.value else []

        padded, ph, pw, _ = pad_mask_for_pbc(mask)
        ap = simulate_aerial_socs(padded, kernels, evals)
        aerial = unpad(ap, ph, pw)
        aerial_np = aerial.cpu().numpy()

        if model.value == 'Mack':
            resist = mack_levelset_resist(aerial, dose.value, dill_c.value,
                                           r_max.value, n_mack.value, steps.value)
            resist_np = resist.cpu().numpy()
        else:
            resist_np = simple_resist(aerial_np, thresh.value, steep.value,
                                      diff.value, load_sig.value, gain.value)

        fig, axes = plt.subplots(1, 3, figsize=(19, 6))
        for ax in axes:
            ax.set_aspect('equal')
            ax.set_xlabel('nm')

        axes[0].imshow(mask.cpu().numpy(), cmap='gray', extent=[0,full_width_nm,full_width_nm,0], origin='lower')
        axes[0].set_title(f"Mask – {sel}{' (inv)' if inv else ''}")
        # Inside update(), where you draw polygons on axes[0] (mask plot)
        for p in polys:
            xmin, ymin, xmax, ymax = [v * pixel_size_nm for v in p]
            
            # Mirror y-coordinates because origin='lower' flipped the axis direction
            y1_flipped = full_width_nm - ymax   # original ymax becomes new bottom
            y2_flipped = full_width_nm - ymin   # original ymin becomes new top
            
            # Now draw rectangle with corrected y-order (y1_flipped < y2_flipped)
            rect = patches.Rectangle(
                (xmin, y1_flipped),           # bottom-left corner
                xmax - xmin,                  # width
                y2_flipped - y1_flipped,      # height
                edgecolor='cyan',
                facecolor='none',
                lw=1.8
            )
            axes[0].add_patch(rect)
        axes[1].imshow(aerial_np, cmap='hot', extent=[0,full_width_nm,full_width_nm,0], origin='lower')
        axes[1].set_title('Aerial')

        im = axes[2].imshow(resist_np, cmap='viridis', vmin=0,vmax=2,
                            extent=[0,full_width_nm,full_width_nm,0], origin='lower')
        axes[2].contour(resist_np>0.5, colors='red', lw=3,
                        extent=[0,full_width_nm,full_width_nm,0], origin='lower')
        axes[2].set_title(f'Resist – {model.value}')
        fig.colorbar(im, ax=axes[2], shrink=0.6)

        sb_nm = 200
        sb_x = full_width_nm - sb_nm - 20
        for ax in axes:
            ax.plot([sb_x, sb_x+sb_nm], [20,20], 'w', lw=4)
            ax.text(sb_x+sb_nm/2, 35, f'{sb_nm} nm', color='w', ha='center', va='bottom', fontsize=9,
                    bbox=dict(fc='black', alpha=0.5, ec='none'))

        plt.tight_layout()
        display(fig)
        plt.close(fig)

btn_update.on_click(update)

display(widgets.VBox([
    widgets.HBox([dd_structure, cb_invert, cb_poly, btn_update]),
    widgets.HBox([model]),
    widgets.HBox([dose, dill_c, r_max, n_mack, steps]),
    widgets.HBox([thresh, steep, diff, load_sig, gain]),
    out
]))

update(None)  # initial plot

In [8]:
# =============================================================================
#  TRAINING DATA GENERATION + INTERACTIVE BROWSER
# =============================================================================

import os
from tqdm.notebook import tqdm

# ──── Settings ───────────────────────────────────────────────────────────────

NUM_SAMPLES       = 200          # change to 5000 or 10000 when ready
HOTSPOT_FRACTION  = 0.10
OUTPUT_PATH       = RAW_TRAINING_DATA_PATH
os.makedirs("data", exist_ok=True)

# Use the same optics as the interactive widget
pixel_size_nm     = 5.0
pad_ratio         = 0.3

# ──── Helpers (copied/adapted from earlier draft) ────────────────────────────

def perturb_to_hotspot(mask):
    mask = mask.clone()
    size = grid_size
    cx, cy = size // 2, size // 2
    num_perturb = np.random.randint(3, 8)
    for _ in range(num_perturb):
        x = np.random.randint(cx - 80, cx + 80)
        y = np.random.randint(cy - 80, cy + 80)
        sz = np.random.randint(5, 20)
        if np.random.rand() < 0.5:
            mask[max(0, y-sz//2):min(size, y+sz//2),
                 max(0, x-sz//2):min(size, x+sz//2)] = 1.0
        else:
            mask[max(0, y-sz//2):min(size, y+sz//2),
                 max(0, x-sz//2):min(size, x+sz//2)] = 0.0
    return mask



# =============================================================================
#  REPLACEMENT: Full set of training structure generators
#  Each returns (mask: torch.Tensor, polygons: list[list[float|int]])
# =============================================================================

def generate_isolated_horizontal_line(size=256, cd_px=12, length_px=180):
    m = torch.zeros((size, size), device=device)
    cy = size // 2
    y1 = cy - cd_px // 2
    y2 = cy + cd_px // 2
    x1 = (size - length_px) // 2
    x2 = x1 + length_px
    m[y1:y2, x1:x2] = 1.0
    return m, [[x1, y1, x2, y2]]


def generate_isolated_vertical_line(size=256, cd_px=12, length_px=180):
    m = torch.zeros((size, size), device=device)
    cx = size // 2
    x1 = cx - cd_px // 2
    x2 = cx + cd_px // 2
    y1 = (size - length_px) // 2
    y2 = y1 + length_px
    m[y1:y2, x1:x2] = 1.0
    return m, [[x1, y1, x2, y2]]


def generate_horizontal_grating(size=256, cd_px=12, pitch_px=28, num_lines=12):
    m = torch.zeros((size, size), device=device)
    polys = []
    cy = size // 2
    start_y = cy - (num_lines * pitch_px) // 2
    for i in range(num_lines):
        y = start_y + i * pitch_px
        if 0 <= y < size - cd_px:
            y1, y2 = y, y + cd_px
            m[y1:y2, :] = 1.0
            polys.append([0, y1, size, y2])
    return m, polys


def generate_vertical_grating(size=256, cd_px=12, pitch_px=28, num_lines=12):
    m = torch.zeros((size, size), device=device)
    polys = []
    cx = size // 2
    start_x = cx - (num_lines * pitch_px) // 2
    for i in range(num_lines):
        x = start_x + i * pitch_px
        if 0 <= x < size - cd_px:
            x1, x2 = x, x + cd_px
            m[:, x1:x2] = 1.0
            polys.append([x1, 0, x2, size])
    return m, polys


def generate_L(size=256, cd_px=12, long_px=140, short_px=80):
    m = torch.zeros((size, size), device=device)
    cx, cy = size // 2, size // 2
    # vertical
    vy1 = cy - long_px // 2
    vy2 = cy + long_px // 2
    vx1 = cx - cd_px // 2
    vx2 = cx + cd_px // 2
    m[vy1:vy2, vx1:vx2] = 1.0
    # horizontal
    hy1 = cy + long_px // 2 - cd_px // 2
    hy2 = cy + long_px // 2 + cd_px // 2
    hx1 = cx - cd_px // 2
    hx2 = cx + short_px
    m[hy1:hy2, hx1:hx2] = 1.0
    return m, [[vx1, vy1, vx2, vy2], [hx1, hy1, hx2, hy2]]


def generate_T(size=256, cd_px=12, stem_px=140, cross_px=100):
    m = torch.zeros((size, size), device=device)
    cx, cy = size // 2, size // 2
    # vertical stem
    vy1 = cy - stem_px // 2
    vy2 = cy + stem_px // 2
    vx1 = cx - cd_px // 2
    vx2 = cx + cd_px // 2
    m[vy1:vy2, vx1:vx2] = 1.0
    # horizontal cross
    hy1 = cy - stem_px // 2 - cd_px // 2
    hy2 = cy - stem_px // 2 + cd_px // 2
    hx1 = cx - cross_px // 2
    hx2 = cx + cross_px // 2
    m[hy1:hy2, hx1:hx2] = 1.0
    return m, [[vx1, vy1, vx2, vy2], [hx1, hy1, hx2, hy2]]


def generate_U(size=256, cd_px=12, height_px=140, width_px=100):
    m = torch.zeros((size, size), device=device)
    cx, cy = size // 2, size // 2
    # left leg
    lx1 = cx - width_px // 2
    lx2 = lx1 + cd_px
    ly1 = cy - height_px // 2
    ly2 = cy + height_px // 2
    m[ly1:ly2, lx1:lx2] = 1.0
    # right leg
    rx1 = cx + width_px // 2 - cd_px
    rx2 = cx + width_px // 2
    m[ly1:ly2, rx1:rx2] = 1.0
    # bottom
    by1 = cy + height_px // 2 - cd_px
    by2 = cy + height_px // 2
    bx1 = cx - width_px // 2
    bx2 = cx + width_px // 2
    m[by1:by2, bx1:bx2] = 1.0
    return m, [[lx1,ly1,lx2,ly2], [rx1,ly1,rx2,ly2], [bx1,by1,bx2,by2]]


def add_symmetric_srafs(mask, center_x, center_y, is_vertical=True,
                        sraf_width=4, num_per_side=2, spacing=30):
    """Add symmetric SRAFs to a vertical or horizontal feature"""
    m = mask.clone()
    polys_add = []
    size = mask.shape[0]
    for side in [-1, 1]:
        for k in range(1, num_per_side + 1):
            offset = k * spacing
            if is_vertical:
                x = center_x + side * offset
                if 0 <= x < size:
                    x1 = x - sraf_width // 2
                    x2 = x + sraf_width // 2
                    m[:, x1:x2] = 1.0
                    polys_add.append([x1, 0, x2, size])
            else:
                y = center_y + side * offset
                if 0 <= y < size:
                    y1 = y - sraf_width // 2
                    y2 = y + sraf_width // 2
                    m[y1:y2, :] = 1.0
                    polys_add.append([0, y1, size, y2])
    return m, polys_add


def generate_line_with_srafs(size=256, cd_px=12, length_px=180, vertical=True):
    if vertical:
        m, polys = generate_isolated_vertical_line(size, cd_px, length_px)
        cx = size // 2
        m, sraf_polys = add_symmetric_srafs(m, cx, None, is_vertical=True)
    else:
        m, polys = generate_isolated_horizontal_line(size, cd_px, length_px)
        cy = size // 2
        m, sraf_polys = add_symmetric_srafs(m, None, cy, is_vertical=False)
    polys.extend(sraf_polys)
    return m, polys


def generate_contacts(size=256, diam_px=14, pitch_px=40, grid=5):
    m = torch.zeros((size, size), device=device)
    polys = []
    cx, cy = size // 2, size // 2
    start_x = cx - (grid * pitch_px) // 2
    start_y = cy - (grid * pitch_px) // 2
    for i in range(grid):
        for j in range(grid):
            x = start_x + i * pitch_px
            y = start_y + j * pitch_px
            if 0 <= x < size and 0 <= y < size:
                x1 = x - diam_px // 2
                x2 = x + diam_px // 2
                y1 = y - diam_px // 2
                y2 = y + diam_px // 2
                m[y1:y2, x1:x2] = 1.0
                polys.append([x1, y1, x2, y2])
    return m, polys


# Dictionary for random selection during dataset generation
training_generators = {
    'isolated_horizontal_line': generate_isolated_horizontal_line,
    'isolated_vertical_line':   generate_isolated_vertical_line,
    'horizontal_grating':       generate_horizontal_grating,
    'vertical_grating':         generate_vertical_grating,
    'L_shape':                  generate_L,
    'T_shape':                  generate_T,
    'U_shape':                  generate_U,
    'line_with_srafs_vertical': lambda: generate_line_with_srafs(vertical=True),
    'line_with_srafs_horizontal': lambda: generate_line_with_srafs(vertical=False),
    'contacts_5x5':             generate_contacts,
}


# Probabilities (you can adjust)
# Weights for each generator (order must match training_generators.keys())
generator_weights = {
    'isolated_horizontal_line': 8,
    'isolated_vertical_line':   8,
    'horizontal_grating':       8,
    'vertical_grating':         8,
    'L_shape':                  9,
    'T_shape':                  8,
    'U_shape':                  7,
    'line_with_srafs_vertical': 8,
    'line_with_srafs_horizontal': 7,
    'contacts_5x5':             7,
    'contacts_with_srafs':      8,
    'jogged_line':              7,
    'hammerhead_line':          7,
}

# Build probs array in correct key order
keys = list(training_generators.keys())
weights_array = [generator_weights.get(k, 1.0) for k in keys]  # 1.0 fallback if missing
training_probs = np.array(weights_array, dtype=np.float64)
training_probs /= training_probs.sum()

print("Probabilities by name:")
for k, p in zip(keys, training_probs):
    print(f"  {k:28s} : {p:.5f}")
print("Sum:", training_probs.sum())
# Or make some more frequent:
# training_probs = [0.12, 0.12, 0.10, 0.10, 0.12, 0.10, 0.08, 0.10, 0.10, 0.06]
# ──── Generate dataset ──────────────────────────────────────────────────────
dataset = {
    'masks':    [],
    'aerials':  [],
    'resists':  [],
    'polygons': [],
    'params':   [],
    'is_hotspot': []
}

print(f"Generating {NUM_SAMPLES} samples...")

dataset = {
    'masks':      [],
    'aerials':    [],
    'resists':    [],
    'polygons':   [],
    'params':     [],
    'is_hotspot': []
}

print(f"Generating {NUM_SAMPLES} samples...")

for i in tqdm(range(NUM_SAMPLES)):
    typ = np.random.choice(list(training_generators.keys()), p=training_probs)
    
    # Get mask + polygons (no params dict returned yet)
    mask, true_polygons = training_generators[typ]()
    
    # Create fresh params dict for this sample
    params = {'type': typ}
    
    # Optional SRAFs on lines & L-variants (add to mask & polygons if needed)
    if typ in ['near_limit_line', 'l_variant', 'isolated_horizontal_line', 'isolated_vertical_line']:
        cx = grid_size // 2
        for s in [-1, 1]:
            for k in [1, 2]:
                x = cx + s * k * 35
                if 0 <= x < grid_size:
                    mask[:, x-3:x+3] = 1.0
                    # Optionally add SRAF polygons here too
                    # true_polygons.append([x-3, 0, x+3, grid_size])

    # Hotspot perturbation
    is_hot = np.random.rand() < HOTSPOT_FRACTION
    if is_hot:
        mask = perturb_to_hotspot(mask)
    
    # Now store the params with hotspot flag
    params['is_hotspot'] = is_hot

    # Compute aerial & resist using current widget settings
    padded, ph, pw, _ = pad_mask_for_pbc(mask)
    aerial_p = simulate_aerial_socs(padded, kernels, evals)
    aerial   = unpad(aerial_p, ph, pw)
    aerial_np = aerial.cpu().numpy()

    if model.value == 'Mack':
        resist = mack_levelset_resist(aerial,
                                      dose=dose.value,
                                      dill_C=dill_c.value,
                                      r_max=r_max.value,
                                      n_mack=n_mack.value,
                                      num_steps=steps.value)
        resist_np = resist.cpu().numpy()
    else:
        resist_np = simple_resist(aerial_np,
                                  thresh=thresh.value,
                                  steep=steep.value,
                                  diff=diff.value,
                                  load_sigma=load_sig.value,
                                  gain=gain.value)

    # Append everything
    dataset['masks'].append(mask.cpu().numpy())
    dataset['aerials'].append(aerial_np)
    dataset['resists'].append(resist_np)
    dataset['polygons'].append(true_polygons)
    dataset['params'].append(params)
    dataset['is_hotspot'].append(is_hot)

# ─── Convert lists to numpy arrays ──────────────────────────────────────────
dataset['masks']    = np.array(dataset['masks'])
dataset['aerials']  = np.array(dataset['aerials'])
dataset['resists']  = np.array(dataset['resists'])
dataset['polygons'] = np.array(dataset['polygons'], dtype=object)
dataset['params']   = np.array(dataset['params'], dtype=object)
dataset['is_hotspot'] = np.array(dataset['is_hotspot'])

# Save
np.savez_compressed(RAW_TRAINING_DATA_PATH, **dataset)
print(f"Dataset saved → {len(dataset['masks'])} samples")

Probabilities by name:
  isolated_horizontal_line     : 0.10256
  isolated_vertical_line       : 0.10256
  horizontal_grating           : 0.10256
  vertical_grating             : 0.10256
  L_shape                      : 0.11538
  T_shape                      : 0.10256
  U_shape                      : 0.08974
  line_with_srafs_vertical     : 0.10256
  line_with_srafs_horizontal   : 0.08974
  contacts_5x5                 : 0.08974
Sum: 1.0
Generating 200 samples...
Generating 200 samples...


  0%|          | 0/200 [00:00<?, ?it/s]

Dataset saved → 200 samples


In [ ]:
# ──── Interactive Dataset Browser ───────────────────────────────────────────

data = np.load(RAW_TRAINING_DATA_PATH, allow_pickle=True)
N = len(data['masks'])
print(f"Loaded dataset with {N} samples from {RAW_TRAINING_DATA_PATH}")

sample_slider = widgets.IntSlider(
    value=0, min=0, max=N-1, step=1,
    description='Sample:', continuous_update=False
)

def view_sample(idx):
    mask    = data['masks'][idx]
    aerial  = data['aerials'][idx]
    resist  = data['resists'][idx]
    polys   = data['polygons'][idx]
    params  = data['params'][idx]
    hotspot = data['is_hotspot'][idx]

    fig, axes = plt.subplots(1, 3, figsize=(19, 6))
    for ax in axes:
        ax.set_aspect('equal')
        ax.set_xlabel('nm')

    # Mask + polygons
    axes[0].imshow(mask, cmap='gray', extent=[0,full_width_nm,full_width_nm,0], origin='lower')
    axes[0].set_title(f"Mask – {params['type']}" + (" 🔥" if hotspot else ""))

    if cb_poly.value and len(polys) > 0:
        for poly in polys:
            try:
                # Expect [xmin, ymin, xmax, ymax]
                if len(poly) == 4 and all(isinstance(v, (int, float)) for v in poly):
                    x1, y1, x2, y2 = [v * pixel_size_nm for v in poly]
                    axes[0].add_patch(patches.Rectangle(
                        (x1, full_width_nm - y2),           # ← y-flip if needed
                        x2 - x1,
                        y2 - y1,
                        ec='cyan', fc='none', lw=1.5
                    ))
                else:
                    print("Skipping malformed polygon:", poly)
            except Exception as e:
                print("Polygon drawing error:", e, "→ poly:", poly)
    # Aerial
    axes[1].imshow(aerial, cmap='hot', extent=[0,full_width_nm,full_width_nm,0], origin='lower')
    axes[1].set_title('Aerial')

    # Resist + contour
    im = axes[2].imshow(resist, cmap='viridis', vmin=0, vmax=2,
                        extent=[0,full_width_nm,full_width_nm,0], origin='lower')
    axes[2].contour(resist > 0.5, colors='red', lw=2.5,
                    extent=[0,full_width_nm,full_width_nm,0], origin='lower')
    axes[2].set_title('Resist + contour')

    fig.colorbar(im, ax=axes[2], shrink=0.6)
    plt.tight_layout()
    plt.show()

widgets.interact(view_sample, idx=sample_slider)

Loaded dataset with 200 samples from data/litho_dataset_training.npz


interactive(children=(IntSlider(value=0, continuous_update=False, description='Sample:', max=199), Output()), …

<function __main__.view_sample(idx)>

In [ ]:
# =============================================================================
#  STAGE 1 – FAST extraction: 10% random patches + batched torch interp
# =============================================================================

import os
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F

# ─── Settings ───────────────────────────────────────────────────────────────

SOURCE_NPZ        = RAW_TRAINING_DATA_PATH     # your 1k images
OUTPUT_CANDIDATES = ALL_PATCHES_DATA_PATH 

PATCH_SIZE         = 48
CENTER_SLICE       = slice(23, 25)
UPSAMPLE_FACTOR    = 10
THRESHOLD          = 0.5
NUM_CLASSES        = 11
SAMPLE_FRAC        = 0.10          # 10% of possible positions
BATCH_SIZE         = 1024          # for torch interpolation

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Device: {device}")

# Load source
print("Loading source dataset...")
data = np.load(SOURCE_NPZ, allow_pickle=True)
N_images = len(data['masks'])
print(f"→ {N_images} images")

# Prepare lists
all_patches      = []
all_resist_avg   = []
all_area_frac    = []
all_source_idx   = []
all_bbox         = []

print("Extracting ~10% random patches per image...")
for img_idx in tqdm(range(N_images), desc="Images"):
    aerial = data['aerials'][img_idx]
    resist = data['resists'][img_idx]
    H, W = aerial.shape

    # Number of samples this image
    num_possible = (H - PATCH_SIZE + 1) * (W - PATCH_SIZE + 1)
    num_sample = int(num_possible * SAMPLE_FRAC)
    if num_sample < 1:
        num_sample = 1

    # Random top-left corners
    rs = np.random.randint(0, H - PATCH_SIZE + 1, size=num_sample)
    cs = np.random.randint(0, W - PATCH_SIZE + 1, size=num_sample)

    # Collect center resist regions for batch interp
    center_resist_list = []

    for r, c in zip(rs, cs):
        patch = aerial[r:r+PATCH_SIZE, c:c+PATCH_SIZE]
        all_patches.append(patch)

        center_res = resist[r+23:r+25, c+23:c+25]
        center_resist_list.append(center_res)

        all_resist_avg.append(center_res.mean())
        all_source_idx.append(img_idx)
        all_bbox.append([r, c, r+PATCH_SIZE, c+PATCH_SIZE])

    # Batch interpolate resist centers
    if center_resist_list:
        batch_np = np.stack(center_resist_list)  # (B,2,2)
        batch_t = torch.from_numpy(batch_np).float().to(device).unsqueeze(1)  # (B,1,2,2)

        up_batch = F.interpolate(
            batch_t,
            scale_factor=UPSAMPLE_FACTOR,
            mode='bilinear',
            align_corners=False
        )  # (B,1,20,20)

        fine_batch = up_batch.squeeze(1).cpu().numpy()  # (B,20,20)

        for fine in fine_batch:
            frac = (fine > THRESHOLD).sum() / fine.size
            all_area_frac.append(frac)

# Convert to arrays
print(f"\nConverting images to arrays")
all_patches      = np.stack(all_patches)[:, None, :, :]
all_resist_avg   = np.array(all_resist_avg, dtype=np.float32)
all_area_frac    = np.array(all_area_frac, dtype=np.float32)
all_source_idx   = np.array(all_source_idx, dtype=np.int64)
all_bbox         = np.array(all_bbox, dtype=np.int64)

# Add classes
print(f"\nAssigning classes")
resist_avg_cls   = np.minimum(NUM_CLASSES-1, np.floor(all_resist_avg * 10 + 1e-9).astype(np.int64))
area_frac_cls    = np.minimum(NUM_CLASSES-1, np.floor(all_area_frac * 10 + 1e-9).astype(np.int64))

# Save
print(f"\nSaving to output file: {OUTPUT_CANDIDATES}")
np.savez_compressed(
    OUTPUT_CANDIDATES,
    patches=all_patches,
    resist_avg=all_resist_avg,
    area_fraction=all_area_frac,
    resist_class=resist_avg_cls,
    area_class=area_frac_cls,
    source_idx=all_source_idx,
    bbox=all_bbox
)

print(f"\nSaved fast candidates: {OUTPUT_CANDIDATES}")
print(f"Total extracted patches: {len(all_patches):,}")
print(f"Resist avg mean/min/max: {all_resist_avg.mean():.4f} / {all_resist_avg.min():.4f} / {all_resist_avg.max():.4f}")
print(f"Area fraction mean/min/max: {all_area_frac.mean():.4f} / {all_area_frac.min():.4f} / {all_area_frac.max():.4f}")

Device: mps
Loading source dataset...
→ 200 images
Extracting ~10% random patches per image...


Images:   0%|          | 0/200 [00:00<?, ?it/s]


Saved fast candidates: data/litho_dataset_all_patches.npz
Total extracted patches: 873,600
Resist avg mean/min/max: 0.1006 / 0.0088 / 0.9998
Area fraction mean/min/max: 0.0844 / 0.0000 / 1.0000


In [12]:
# =============================================================================
#  STAGE 2 – Fast stratified sampling using per-class lists + final save
# =============================================================================

FINAL_NPZ = SAMPLED_PATCHES_DATA_PATH
N_TOTAL   = 100_000
N_RANDOM  = N_TOTAL // 2
N_STRAT   = N_TOTAL - N_RANDOM
QUOTA_PER_CLASS = N_STRAT // NUM_CLASSES

print(f"Target: {N_TOTAL:,} patches")
print(f"  → {N_RANDOM:,} random")
print(f"  → {N_STRAT:,} stratified (~{QUOTA_PER_CLASS:,} per class)")

# Load candidates from Stage 1
print("Loading candidates from Stage 1...")
candidates = np.load(OUTPUT_CANDIDATES, allow_pickle=True)

patches_all     = candidates['patches']
resist_avg_all  = candidates['resist_avg']
area_frac_all   = candidates['area_fraction']
source_all      = candidates['source_idx']
bbox_all        = candidates['bbox']

N_all = len(patches_all)
print(f"→ {N_all:,} total candidates")

# Compute classes (using resist_avg for balancing — you can switch to area_frac if preferred)
resist_avg_cls = np.minimum(NUM_CLASSES-1, np.floor(resist_avg_all * 10 + 1e-9).astype(np.int64))

# Build per-class index lists
print("Building per-class index lists...")
indices_per_class = [[] for _ in range(NUM_CLASSES)]

for i in tqdm(range(N_all), desc="Classifying"):
    cls = resist_avg_cls[i]
    indices_per_class[cls].append(i)

raw_counts = [len(lst) for lst in indices_per_class]
print("Raw class counts (resist avg):", raw_counts)

# ─── Random half ────────────────────────────────────────────────────────────
print("\nSampling random half...")
random_indices = np.random.choice(N_all, size=N_RANDOM, replace=False)

# ─── Stratified half – direct sampling per class ────────────────────────────
print("Sampling balanced stratified half...")
strat_indices = []

for cls in range(NUM_CLASSES):
    avail = indices_per_class[cls]
    if len(avail) == 0:
        print(f"Warning: class {cls} has zero candidates!")
        continue
    
    n_take = min(QUOTA_PER_CLASS, len(avail))
    chosen = np.random.choice(avail, size=n_take, replace=False)
    strat_indices.extend(chosen)

    print(f"  Class {cls}: took {n_take:,} / {QUOTA_PER_CLASS:,} (available: {len(avail):,})")

strat_indices = np.array(strat_indices)

# If we got fewer than N_STRAT due to rare classes, fill with random
if len(strat_indices) < N_STRAT:
    remaining = N_STRAT - len(strat_indices)
    extra_random = np.random.choice(N_all, size=remaining, replace=False)
    strat_indices = np.concatenate([strat_indices, extra_random])
    print(f"Added {remaining:,} extra random to reach target stratified count")

# ─── Combine & save final dataset ───────────────────────────────────────────
print("\nCombining random + stratified...")

final_indices = np.concatenate([random_indices, strat_indices])

final_patches     = patches_all[final_indices]
final_resist_avg  = resist_avg_all[final_indices]
final_area_frac   = area_frac_all[final_indices]
final_resist_cls  = resist_avg_cls[final_indices]
final_area_cls    = np.minimum(NUM_CLASSES-1, np.floor(final_area_frac * 10 + 1e-9).astype(np.int64))
final_source      = source_all[final_indices]
final_bbox        = bbox_all[final_indices]
final_is_random   = np.concatenate([np.ones(N_RANDOM, dtype=bool), np.zeros(len(strat_indices), dtype=bool)])

np.savez_compressed(
    FINAL_NPZ,
    patches            = final_patches.astype(np.float32),
    resist_avg         = final_resist_avg.astype(np.float32),
    area_fraction      = final_area_frac.astype(np.float32),
    resist_class       = final_resist_cls.astype(np.int64),
    area_class         = final_area_cls.astype(np.int64),
    source_idx         = final_source.astype(np.int64),
    patch_bbox         = final_bbox.astype(np.int64),
    is_random          = final_is_random,
    class_hist_random  = np.bincount(final_resist_cls[:N_RANDOM], minlength=NUM_CLASSES),
    class_hist_strat   = np.bincount(final_resist_cls[N_RANDOM:], minlength=NUM_CLASSES),
    class_hist_total   = np.bincount(final_resist_cls, minlength=NUM_CLASSES)
)

print(f"\nFinal dataset saved: {FINAL_NPZ}")
print(f"Total patches: {len(final_patches):,}")
print("Class distribution (resist avg):")
print(np.bincount(final_resist_cls, minlength=NUM_CLASSES))
print("\nRandom half class counts:")
print(np.bincount(final_resist_cls[:N_RANDOM], minlength=NUM_CLASSES))
print("Stratified half class counts:")
print(np.bincount(final_resist_cls[N_RANDOM:], minlength=NUM_CLASSES))

Target: 100,000 patches
  → 50,000 random
  → 50,000 stratified (~4,545 per class)
Loading candidates from Stage 1...
→ 873,600 total candidates
Building per-class index lists...


Classifying:   0%|          | 0/873600 [00:00<?, ?it/s]

Raw class counts (resist avg): [722845, 33333, 17321, 13278, 13838, 12793, 15322, 19469, 13756, 11645, 0]

Sampling random half...
Sampling balanced stratified half...
  Class 0: took 4,545 / 4,545 (available: 722,845)
  Class 1: took 4,545 / 4,545 (available: 33,333)
  Class 2: took 4,545 / 4,545 (available: 17,321)
  Class 3: took 4,545 / 4,545 (available: 13,278)
  Class 4: took 4,545 / 4,545 (available: 13,838)
  Class 5: took 4,545 / 4,545 (available: 12,793)
  Class 6: took 4,545 / 4,545 (available: 15,322)
  Class 7: took 4,545 / 4,545 (available: 19,469)
  Class 8: took 4,545 / 4,545 (available: 13,756)
  Class 9: took 4,545 / 4,545 (available: 11,645)
Added 4,550 extra random to reach target stratified count

Combining random + stratified...

Final dataset saved: data/litho_dataset_sampled_patches.npz
Total patches: 100,000
Class distribution (resist avg):
[49848  6536  5643  5371  5356  5329  5496  5726  5381  5314     0]

Random half class counts:
[41551  1833  1000   747   